In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [6]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

In [8]:
df_cities_skylines = spark \
    .read \
        .parquet('file:///home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/assunto_cities_skylines/extracao_data_2023_10_15/estatisticas/historico_video_tratada.parquet')

AnalysisException: Path does not exist: file:/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/assunto_cities_skylines/extracao_data_2023_10_15/estatisticas/historico_video_tratada.parquet

In [33]:
df_cities_skylines.show(truncate=False)

+-------------------+-------------+----------------------+--------------------------+------------------------------+------------+------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
from pyspark.sql.functions import unix_timestamp, date_format


In [66]:

df_cities_skylines = df_cities_skylines.select(
    'data_extracao', 
    'NM_CANAL',
    'ID_CANAL',
     f.explode('ID_VIDEO').alias('ID_VIDEO')
     ) \
        .select(
            'data_extracao', 
            'ID_VIDEO',
            'ID_CANAL',
            f.explode('NM_CANAL').alias('NM_CANAL')
        )  \
            .select(
            'data_extracao', 
            'ID_VIDEO',
            'NM_CANAL',
            f.explode('ID_CANAL').alias('ID_CANAL')
        )  

       
df_cities_skylines = df_cities_skylines.withColumn("data_extracao", date_format(unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))
     
df_cities_skylines.show(truncate=False)

+-------------+-----------+----------------------------+------------------------+
|data_extracao|ID_VIDEO   |NM_CANAL                    |ID_CANAL                |
+-------------+-----------+----------------------------+------------------------+
|2023-10-26   |2qDGrXMCHPE|mkjxzz                      |UCt0IBPkhrVM43xSyP2vCnAw|
|2023-10-26   |TI_4qn0KEUs|Pinguires                   |UCla_gQERStTS74gxHMS6Tzw|
|2023-10-26   |7NWFw5SiqRE|Betospa Gaming              |UCkXA2EoaXRFW18vrAE01BpQ|
|2023-10-26   |GaYWC83B-uw|CANAL HAND - GAME WITH MUSIC|UC7v51W2NoLFawizgcNuVBOA|
|2023-10-26   |gYl77Ot9oDU|Octopolis                   |UCCe4Be21OPPltTTIHP4lDvg|
|2023-10-26   |End_beEw42E|Grilo Gamer                 |UCwBN6GQiolq3EkRPYC7OTtA|
|2023-10-26   |ZCPbAwRwv5o|wzz                         |UCLXbP3xz78Yz7HW5avm4zDg|
|2023-10-26   |3SsUx-y5Ik0|Ubira                       |UCHuvObnh1-2fs6zL5nhVGtg|
|2023-10-26   |apbsKuImj5w|BUG CITIES                  |UCDqzBrUOm7guqqyU8eDackg|
|2023-10-26   |o

In [9]:
df_cities_skylines.show()

+-------------+-------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+
|data_extracao|     ID_VIDEO|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|
+-------------+-------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+
|   2023-10-26|[2qDGrXMCHPE]|[2023-10-24T00:45...|[UCt0IBPkhrVM43xS...|            [mkjxzz]|        [20]|[mkj ( Fivem High...|[================...|              [null]|     [PT1M16S]|               [87]|        [8]|            [0]|             [17]|


In [56]:
df_cities_skylines.select('data_extracao').distinct().show()

+-------------+
|data_extracao|
+-------------+
|   2023-10-26|
|   2023-10-25|
|   2023-10-23|
|   2023-10-24|
|   2023-10-22|
|   2023-10-21|
|   2023-10-19|
|   2023-10-20|
|   2023-10-18|
|   2023-10-17|
|   2023-10-15|
|   2023-10-16|
+-------------+



In [67]:
df_cities_skylines \
    .write\
    .partitionBy('data_extracao', 'ID_CANAL')\
        .mode('overwrite').\
            parquet('/home/rodrigo/projetos/analise_dados_youtube/data/projetos_youtube/ouro/estatistica/cities_skylines.parquet')